In [1]:
import maidenhead
import datetime
import pandas as pd
import dask.dataframe as dadf
import glob
import pyarrow.parquet as pq
import pyarrow as pa
import tqdm
import easier as ezr
# import datashader.spatial.points as dsp

In [2]:
available_cols = [
    'spot_id',
    'timestamp',
    'reporter',
    'reporters_grid',
    'snr',
    'frequency',
    'call_sign',
    'grid',
    'power',
    'drift',
    'distance',
    'azimuth',
    'band',
    'version',
    'code'
]

ddf = dadf.read_csv('./data/wsprnet/2020/wsprspots-2020-01_*.csv.gz', compression='gzip', names=available_cols, blocksize=None)
ddf = ddf.set_index('timestamp')
out_file = './data/wsprnet/parquet_2020_raw/'

ddf.head()


,spot_id,reporter,reporters_grid,snr,frequency,call_sign,grid,power,drift,distance,azimuth,band,version,code
timestamp,,,,,,,,,,,,,,
1577836800,1913207476,DP0GVN,IB59uh,-23,7.040077,2E0ETU,IO81wv,27,0,13640,182,7,NaN,0
1577836800,1913208199,IW2NKE,JN63np,-10,7.040077,2E0ETU,IO81wv,27,0,1458,123,7,NaN,0
1577836800,1913208025,IZ3EAW/B,JN55xi,-20,7.040029,2E0ETU,IO81wv,27,0,1262,120,7,1.3 Kiwi,0
1577836800,1913206488,KF4TEK,FM18dr,-22,7.040079,2E0ETU,IO81wv,27,0,5810,287,7,2.1.2,0
1577836800,1913208961,OE9HLH,JN47um,-25,7.040077,2E0ETU,IO81wv,27,0,978,115,7,NaN,0


In [ ]:
ddf.to_parquet(out_file, write_index=True)
ddf.head()

In [1]:
ddf.head()

NameError: name 'ddf' is not defined

In [2]:
def get_frame(file_name):
    available_cols = [
        'spot_id',
        'timestamp',
        'reporter',
        'reporters_grid',
        'snr',
        'frequency',
        'call_sign',
        'grid',
        'power',
        'drift',
        'distance',
        'azimuth',
        'band',
        'version',
        'code'
    ]

    desired_cols = [
        'spot_id',
        'timestamp',
        'reporter',
        'reporters_grid',
        'snr',
        'frequency',
        'call_sign',
        'grid',
        'power',
        'distance',
        'code',
    ]


    col_numbers = [available_cols.index(v) for v in desired_cols]
    df = pd.read_csv(
        file_name, 
        usecols=col_numbers,
        names=available_cols,
        compression='gzip',
    )
    df['lat_reporter'], df['lon_reporter'] = list(zip(*(maidenhead.to_location(m) for m in df.reporters_grid)))
    df['lat'], df['lon'] = list(zip(*(maidenhead.to_location(m) for m in df.grid)))
    df = df[df.code == 0]
    
    df['time'] = df.timestamp // 3600

    df = df.groupby(by=['time', 'call_sign', 'reporter', 'lat', 'lon', 'lat_reporter', 'lon_reporter', 'distance'])[['power', 'snr']].mean()
    df = df.reset_index()
    df['time_local'] = [t + h for (t, h) in zip(df.time, df.lon * 12 / 180)]

#     df = df.set_index('time').sort_index()
    return df

file_names = sorted(glob.glob('./data/wsprnet/2020/*csv.gz'))[:50]

out_file = './data/wsprnet/parquet_2020_no_index/'


if True:
    for in_file in tqdm.tqdm(file_names):
        df = get_frame(in_file)
        table = pa.Table.from_pandas(df)
        pq.write_to_dataset(table, root_path=out_file)



In [3]:
dff = dadf.read_parquet(out_file, engine='pyarrow', sorted_index=False)
# dff = dff.loc[pd.Timestamp('7/1/2020'): pd.Timestamp('7/4/2020')]
# dff = dff.reset_index()
# dff['day'] = dff.time_local.dt.floor('D')
# ser = dff.day.value_counts().compute()
# dfx = dff.compute()
dff.head()

,call_sign,reporter,lat,lon,lat_reporter,lon_reporter,distance,power,snr,time_local
time,,,,,,,,,,
439477,0C4QQD,SWLKCL,0.0,92.0,43.0,-80.0,15063,10.0,-7.0,439483.133333
439477,0C4QQE,SWLKCL,9.0,92.0,43.0,-80.0,14071,0.0,-8.0,439483.133333
439477,0C4QQE,SWLKCL,31.0,118.0,43.0,-80.0,11481,7.0,2.0,439484.866667
439477,0C4QQE,SWLKCL,43.0,82.0,43.0,-80.0,10181,53.0,-11.0,439482.466667
439477,0C4QQE,SWLKCL,77.0,92.0,43.0,-80.0,6554,33.0,-11.0,439483.133333


In [ ]:
with ezr.Timer('sorting'):
    out_file_sorted = './data/wsprnet/parquet_2020_sorted/'
    print('reading')
    dff2 = dadf.read_parquet(out_file, engine='pyarrow')
    print('resetting')
    dff2 = dff2.reset_index()
    print('setting')
    dff2 = dff2.set_index('time')
    print('saving')
    yyy = dff2.to_parquet(out_file_sorted, engine='pyarrow')

reading
resetting
setting
saving


In [ ]:
print('ehhlo')

In [5]:
ind = dff.index.compute()

In [8]:
s = pd.Series(ind.values)

In [13]:
dd = s.diff().dropna()
dd = dd[dd != 0.0]

In [15]:
dd.min()

-5860.0

In [ ]:
df['hour'] = df.timestamp // 3600
df['time'] = [epoch + datetime.timedelta(seconds=t) for t in df.timestamp]
df['hour'] = [fleming.floor(t, hour=1) for t in df.time]
df['rg'] = df.reporters_grid.str[:2]
df['g'] = df.grid.str[:2]
df['l1'] = df.call_sign.str[:1]
df['l2'] = df.reporter.str[:1]
df['fl'] = [frozenset(tup) for tup in zip(df.l1, df.l2)]
# df['pair'] = [frozenset(tup) for tup in zip(df.reporter, df.call_sign)]
# df['pair'] = [frozenset(tup) for tup in zip(df.reporters_grid, df.grid)]
df['pair'] = [frozenset(tup) for tup in zip(df.rg, df.g)]
df['us'] = [frozenset(s.intersection({'K', 'W', 'A'})) for s in df.fl]
df['us_size'] = [len(s) for s in df.us]
# df = df[df.us_size > 0]
df['hr'] = df.hour.dt.hour

lat, lon = list(zip(*(maidenhead.to_location(m) for m in df.reporters_grid)))
df['lat_reporter'] = lat
df['lon_reporter'] = lon

lat, lon = list(zip(*(maidenhead.to_location(m) for m in df.grid)))
df['lat'] = lat
df['lon'] = lon
df['time_local'] = [t + datetime.timedelta(hours=h) for (t, h) in zip(df.time, df.lon * 12 / 180)]
df['hour_local'] = [fleming.floor(t, hour=1) for t in df.time_local]
df['hr_local']= df.hour_local.dt.hour

print(len(df))